# Drinking Water Quality in NYC ETL.
---

### Summary

- This is code for extracting and transforming a dataset from NYC dept __________ and loading the dataset into SQL.

### Extract

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2


# Study data files
water_path = "Data/drinking-water-quality-distribution-monitoring-data.csv"

# Read imported csv
water = pd.read_csv(water_path)

# Display the data table for preview
water.head()

C:\Users\bryan\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3186: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Sample Number,Sample Date,Sample Time,Sample Site,Sample class,Location,Residual Free Chlorine (mg/L),Turbidity (NTU),Fluoride (mg/L),Coliform (Quanti-Tray) (MPN /100mL),E.coli(Quanti-Tray) (MPN/100mL)
0,32355.0,2019-10-31T00:00:00.000,8:23,24950,Compliance,"SS - IFO 1169 N/S East New York Ave, 2nd SS W/...",0.42,0.61,NaN,<1,<1
1,32320.0,2019-10-31T00:00:00.000,11:14,13550,Compliance,"SS - IFO 2015 W/S University Ave, 1st SS S/O W...",0.45,0.59,NaN,<1,<1
2,32357.0,2019-10-31T00:00:00.000,11:33,50200,Operational,"SS - IFO 93 N/S Austin Place, BTW Victory Blvd...",0.74,0.63,NaN,<1,<1
3,32353.0,2019-10-31T00:00:00.000,7:47,24650,Compliance,"SS - N/S Linden Blvd, 1st SS E/O Bristol St, 12 """,0.43,0.68,NaN,1,<1
4,32371.0,2019-10-31T00:00:00.000,9:32,17050,Compliance,"SS - IFO 4740 E/S White Plains Rd, 1st SS N/O ...",0.51,0.55,NaN,<1,<1


### Transform 

In [2]:
#Modifying the Dataset to include only datapoints that are collected in 2019 

#Checking number of samples in current dataset
number_of_samples = len(water["Sample Number"].unique())
print(f"Number of samples in original dataset is {number_of_samples}")

#Converting Sample Date to datetime64
water["Sample Date"] = pd.to_datetime(water["Sample Date"], format='%Y-%m-%d')

#Inlcuding only samples collected in 2019
water_date = water.loc[water["Sample Date"] >= '2019-01-01']

#Checking number of samples in revised dataset
number_of_samples_2019 = len(water_date["Sample Number"].unique())
print(f"Number of samples in modified dataset is {number_of_samples_2019}")


Number of samples in original dataset is 41987
Number of samples in modified dataset is 13303


In [3]:
#Removing duplicates and extraneous information
#Show all duplicates in dataset
water_date[water_date.duplicated()]

,Sample Number,Sample Date,Sample Time,Sample Site,Sample class,Location,Residual Free Chlorine (mg/L),Turbidity (NTU),Fluoride (mg/L),Coliform (Quanti-Tray) (MPN /100mL),E.coli(Quanti-Tray) (MPN/100mL)
10000,7807.0,2019-03-18,11:26,50200,Operational,"SS - IFO 93 N/S Austin Place, BTW Victory Blvd...",0.55,0.98,NaN,<1,<1
10001,7790.0,2019-03-18,10:28,22600,Operational,"SS - E/S Ft Hamilton Pkwy, BTW 49th & 50th Sts...",0.44,0.97,NaN,<1,<1
10014,7761.0,2019-03-18,11:52,40200,Operational,"SS - S/S 19th Ave, BTW 41st & Steinway Sts, IF...",0.72,1.11,NaN,<1,<1
10023,7809.0,2019-03-18,12:01,50300,Operational,"SS - N/E/S Clove Rd, N/W of Sunnyside Terrace.",0.80,1.08,NaN,<1,<1
10024,7808.0,2019-03-18,11:44,50250,Compliance,"SS - IFO 937 N/S Victory Blvd, 2nd SS E/O High...",0.58,1.11,NaN,<1,<1


In [4]:
#Removing all duplicates and showing there is no duplicates in dataset
water_clean = water_date.drop_duplicates()
water_clean[water_clean.duplicated()]

,Sample Number,Sample Date,Sample Time,Sample Site,Sample class,Location,Residual Free Chlorine (mg/L),Turbidity (NTU),Fluoride (mg/L),Coliform (Quanti-Tray) (MPN /100mL),E.coli(Quanti-Tray) (MPN/100mL)


In [5]:
#Deleting extraneous information
water_2019 = water_clean.drop(columns=["Sample Time", "Sample class"])
water_2019.head()

,Sample Number,Sample Date,Sample Site,Location,Residual Free Chlorine (mg/L),Turbidity (NTU),Fluoride (mg/L),Coliform (Quanti-Tray) (MPN /100mL),E.coli(Quanti-Tray) (MPN/100mL)
0,32355.0,2019-10-31,24950,"SS - IFO 1169 N/S East New York Ave, 2nd SS W/...",0.42,0.61,NaN,<1,<1
1,32320.0,2019-10-31,13550,"SS - IFO 2015 W/S University Ave, 1st SS S/O W...",0.45,0.59,NaN,<1,<1
2,32357.0,2019-10-31,50200,"SS - IFO 93 N/S Austin Place, BTW Victory Blvd...",0.74,0.63,NaN,<1,<1
3,32353.0,2019-10-31,24650,"SS - N/S Linden Blvd, 1st SS E/O Bristol St, 12 """,0.43,0.68,NaN,1,<1
4,32371.0,2019-10-31,17050,"SS - IFO 4740 E/S White Plains Rd, 1st SS N/O ...",0.51,0.55,NaN,<1,<1


In [6]:
#Replacing NaN values. 
water_2019["Turbidity (NTU)"] = water_2019["Turbidity (NTU)"].fillna(0)
water_2019["Fluoride (mg/L)"] = water_2019["Fluoride (mg/L)"].fillna(0)

water_2019.head()

,Sample Number,Sample Date,Sample Site,Location,Residual Free Chlorine (mg/L),Turbidity (NTU),Fluoride (mg/L),Coliform (Quanti-Tray) (MPN /100mL),E.coli(Quanti-Tray) (MPN/100mL)
0,32355.0,2019-10-31,24950,"SS - IFO 1169 N/S East New York Ave, 2nd SS W/...",0.42,0.61,0,<1,<1
1,32320.0,2019-10-31,13550,"SS - IFO 2015 W/S University Ave, 1st SS S/O W...",0.45,0.59,0,<1,<1
2,32357.0,2019-10-31,50200,"SS - IFO 93 N/S Austin Place, BTW Victory Blvd...",0.74,0.63,0,<1,<1
3,32353.0,2019-10-31,24650,"SS - N/S Linden Blvd, 1st SS E/O Bristol St, 12 """,0.43,0.68,0,1,<1
4,32371.0,2019-10-31,17050,"SS - IFO 4740 E/S White Plains Rd, 1st SS N/O ...",0.51,0.55,0,<1,<1


In [7]:
#Changing column names. Dataframe is finally cleaned and ready to go.
water_2019.rename(columns = {"Residual Free Chlorine (mg/L)":"Residual Free Chlorine mg per L", \
                             "Turbidity (NTU)":"Turbidity in NTU", \
                             "Fluoride (mg/L)":"Fluoride in mg per L", \
                             "Coliform (Quanti-Tray) (MPN /100mL)":"Coliform_Quanti-Tray in MPN per 100mL", \
                             "E.coli(Quanti-Tray) (MPN/100mL)":"E.coli_Quanti-Tray in MPN per 100mL", 
                            }, inplace = True)
water_2019.head()

,Sample Number,Sample Date,Sample Site,Location,Residual Free Chlorine mg per L,Turbidity in NTU,Fluoride in mg per L,Coliform_Quanti-Tray in MPN per 100mL,E.coli_Quanti-Tray in MPN per 100mL
0,32355.0,2019-10-31,24950,"SS - IFO 1169 N/S East New York Ave, 2nd SS W/...",0.42,0.61,0,<1,<1
1,32320.0,2019-10-31,13550,"SS - IFO 2015 W/S University Ave, 1st SS S/O W...",0.45,0.59,0,<1,<1
2,32357.0,2019-10-31,50200,"SS - IFO 93 N/S Austin Place, BTW Victory Blvd...",0.74,0.63,0,<1,<1
3,32353.0,2019-10-31,24650,"SS - N/S Linden Blvd, 1st SS E/O Bristol St, 12 """,0.43,0.68,0,1,<1
4,32371.0,2019-10-31,17050,"SS - IFO 4740 E/S White Plains Rd, 1st SS N/O ...",0.51,0.55,0,<1,<1


### Load

In [8]:
#Loading into PostgresSQL
rds_connection_string = "postgres:postgres@localhost:5432/water_2019"
engine = create_engine(f'postgresql://{rds_connection_string}')
conn = engine.connect()
print("Database is loaded into PostgresSQL")

Database is loaded into PostgresSQL


In [9]:
water_2019.to_sql(name='water_2019',con=engine, if_exists='append', index=False)
pd.read_sql_query('select * from water_2019', conn).head()

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "Residual Free Chlorine mg per L" of relation "water_2019" does not exist
LINE 1: ...umber", "Sample Date", "Sample Site", "Location", "Residual ...
                                                             ^

[SQL: INSERT INTO water_2019 ("Sample Number", "Sample Date", "Sample Site", "Location", "Residual Free Chlorine mg per L", "Turbidity in NTU", "Fluoride in mg per L", "Coliform_Quanti-Tray in MPN per 100mL", "E.coli_Quanti-Tray in MPN per 100mL") VALUES (%(Sample Number)s, %(Sample Date)s, %(Sample Site)s, %(Location)s, %(Residual Free Chlorine mg per L)s, %(Turbidity in NTU)s, %(Fluoride in mg per L)s, %(Coliform_Quanti-Tray in MPN per 100mL)s, %(E.coli_Quanti-Tray in MPN per 100mL)s)]
[parameters: ({'Sample Number': 32355.0, 'Sample Date': datetime.datetime(2019, 10, 31, 0, 0), 'Sample Site': '24950', 'Location': 'SS - IFO 1169 N/S East New York Ave, 2nd SS W/O Ralph Ave, 12 "', 'Residual Free Chlorine mg per L': 0.